In [2]:
def deco(func):
    def inner():
        print 'running inner()'
    return inner

@deco
def target():
    print 'running target()'

In [3]:
target()  # 데코레이트된 함수 target()을 호출하면 inner()를 실행한다.

running inner()


In [4]:
target  # target 함수는 inner함수를 가리키고 있다.

<function __main__.inner>

데코레이터는 데코레이트된 함수를 다른 함수로 대체할 수 있다.

### 파이썬이 데코레이터를 실행하는 시점

데코레이터의 핵심 특징은 데코레이트된 함수가 정의된 직후에 바로 실행된다는 점이다.
이는 파이썬이 모듈을 로딩하는 시점, 즉 import 타임에 실행된다.

In [14]:
registry = []
def register(func):
    print 'running register. paramter is {}'.format(func)
    registry.append(func)
    return func

@register
def func1():
    print 'running func1()'
    
@register
def func2():
    print 'running func2()'
    
def func3():
    print 'running func3()'
    
def main():
    print 'running main()'
    print 'registry: {}'.format(registry)
    func1()
    func2()
    func3()

if __name__ == '__main__':
    main()

running register. paramter is <function func1 at 0x7fd17743e398>
running register. paramter is <function func2 at 0x7fd17743e6e0>
running main()
registry: [<function func1 at 0x7fd17743e398>, <function func2 at 0x7fd17743e6e0>]
running func1()
running func2()
running func3()


`register()` 데코레이터는 모듈에서 가장 먼저 실행된다. func1 ~ func3 함수는 `main`에서 명시적으로 호출될 때만 실행된다. <br>
위 코드를 `registration.py`모듈에 저장하고 모듈을 로딩하면,

In [16]:
import registration

running register. paramter is <function func1 at 0x7fd17743e398> <br>
running register. paramter is <function func2 at 0x7fd17743e6e0>

데코레이터는 모듈이 import 되자마자 실행되지만, 함수는 명시적으로 호출될 때만 실행됨을 알 수 있다.<br>
이는 파이썬에서 임포트타임과 런타임을 구분하는 이유이기도 하다. <br>
> `register()`의 경우 파라미터로 받은 함수를 그대로 리턴하지만, 보통의 데코레이터는 내부에서 새롭게 정의한 함수를 리턴한다.

### 변수 범위 규칙

In [20]:
b = 6  # 전역변수
def test_local_var(a):
    print a
    print b  # 6을 출력할까?
    b = 9
    

In [21]:
test_local_var(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

전역변수 `b`가 이미 정의되어 있기 때문에 `test_local_var`에서 6을 출력할 것으로 예상했지만 결과는 `UnboundLocalError`가 발생했다.<br>
왜 그럴까?

In [22]:
from dis import dis
dis(test_local_var)

  3           0 LOAD_FAST                0 (a)
              3 PRINT_ITEM          
              4 PRINT_NEWLINE       

  4           5 LOAD_FAST                1 (b)
              8 PRINT_ITEM          
              9 PRINT_NEWLINE       

  5          10 LOAD_CONST               1 (9)
             13 STORE_FAST               1 (b)
             16 LOAD_CONST               0 (None)
             19 RETURN_VALUE        


python3에서 디스어셈블 출력한 결과

In [24]:
  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (6)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


SyntaxError: invalid syntax (<ipython-input-24-329d15ace98e>, line 1)

`10    LOAD_FAST`: 지역변수 `b`를 로딩한다. <br>
4번의 어셈블리어 코드를 보면 지역변수 `b`을 6으로 정의하는 부분이 나오지만, 컴파일러는 이미 이전에 `b`를 지역변수로 간주하고 있다. 

### 클로저

클로저가 뭔지 알아보기 전에 클래스를 이용해서 예제를 만들어보겠다. <br>
아래 예제는 새로운 숫자가 추가 될 때마다 추가된 숫자를 포함하여 숫자들의 평균을 계산한다.

In [32]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):  # 객체를 call할 때 호출되는 함수.
        self.series.append(new_value)
        total = sum(self.series)
        return total // len(self.series)

In [38]:
avg = Averager()
print avg(10)
print avg(11)
print avg(12)
# 파이썬2에서는 나누기 결과가 실수일 경우 내림으로 처리.

10
10
11


다음은 고차함수 `make_averager()`를 이용해서 예제를 구현한 것이다.

In [82]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/ len(series)
    return averager

In [83]:
avg = make_averager()  # make_averager 함수 안의 averager 함수
print avg(10)
print avg(12)
print avg(14)

10
11
12


`make_averager()` 함수 안의 `series`는 지역변수이므로 `make_averager`가 호출되고 `averager`를 리턴하고 나면 사라져야 마땅하다. <br>
그런데 `make_averager()`가 호출될때마다 이전에 숫자들이 추가된 `series`를 참조하여 숫자들의 평균을 구하게 된다. 왜 그럴까?

왜냐하면 `averager()` 안의 `series`는 **자유변수**기 때문이다.<br>
**자유변수는 지역범위에 바인딩 되어있지 않은 변수를 뜻한다.**

컴파일된 함수를 나타내는 `__code__`속성을 들여다보면 지역변수와 자유변수의 이름들을 알아낼 수 있다.

In [84]:
print avg.__code__.co_varnames  # 지역변수 이름들
print avg.__code__.co_freevars    # 자유변수 이름들

('new_value', 'total')
('series',)


`series`에 대한 바인딩은 반환된 `averager`함수의 `__closure__`속성에 리스트형태로 저장된다. <br>
`__closure__`의 각 엘리먼트는 `cell`객체이며, `cell`객체의 `cell_contents`속성에서 자유변수의 실제 값을 찾을 수 있다. <br>

> cells are special references to local variables of a parent scope, that follow the values those local variables point to. https://stackoverflow.com/questions/14413946/what-exactly-is-contained-within-a-obj-closure

In [85]:
print avg.__closure__[0].cell_contents

[10, 12, 14]


위의 예제에 따르면 **클로저는 어떤 함수안에 정의된 함수며, 어떤 함수를 정의할 때 존재하던 자유변수의 바인딩을 유지해주는 함수라고 할 수 있다.**

클로저와 자유변수를 이용하여 `decorate`된 함수를 `undecorate` 해보도록 하겠다. <br>
`decorators`모듈의 `check_now_is_in_period`함수는 무조건 `out_of_period`문자열을 리턴하는 데코레이터이며,<br>
`is_power_of_two`함수는 파라미터로 들어온 숫자가 2의 n(자연수)제곱에 해당하는 숫자인지 판단한다.

In [28]:
from datetime import datetime, timedelta
from functools import wraps

# decorators.py
def check_now_is_in_period(func):
    def wrapper(*args, **kwargs):
            start = datetime.today() - timedelta(days=2)
            end = datetime.today() - timedelta(days=1)
            if start <= datetime.now() <= end:
                return func(*args, **kwargs)
            return 'out of period'
    return wrapper

In [30]:
from inspect import isfunction
from decorators import check_now_is_in_period


@check_now_is_in_period
def is_power_of_two(num):
    while(num != 1):
        if num % 2 != 0:
            return False
        num = num // 2
    return True


def undecorate(func):
    if func.__closure__:
        for cell in func.__closure__:
            if isfunction(cell.cell_contents):
                return cell.cell_contents
        return func


if __name__ == '__main__':
    p = is_power_of_two
    print 'local vars: ', p.__code__.co_varnames
    print 'free vars: ', p.__code__.co_freevars
    print 'closure: ', p.__closure__
    print "closure's cell content: ", p.__closure__[0].cell_contents
    print undecorate(is_power_of_two)(32)
    print undecorate(is_power_of_two)(10)                                             

local vars:  ('args', 'kwargs', 'start', 'end')
free vars:  ('func',)
closure:  (<cell at 0x7f2a702b2830: function object at 0x7f2a7046a410>,)
closure's cell content:  <function is_power_of_two at 0x7f2a7046a410>
True
False


`is_power_of_two`함수는 모듈이 로딩되는 시점에서 이미 `check_now_is_in_period`함수에 의해 `decorate`되었으므로<br>
`check_now_is_in_period`함수에서 리턴된 `wrapper`함수와 동일한 지역변수와 자유변수를 갖는다. (동일한 환경을 갖는다.)<br>
`wrapper`함수의 `__closure__`에는 parental scope(`check_now_is_in_period`)의 변수(`func`)가 들어있다.<br>
따라서 `is_power_of_two`의 `__closure__`에도 `func` 파라미터에 해당하는 `is_power_of_two`, 즉 자기자신을 cell 객체로 가지게 된다. <br><br>
`undecorate`함수는 파라미터로 들어오는 함수의 cell객체의 `cell_contents`가 함수일 경우 `cell_contents`를 리턴한다.
`is_power_of_two`의 `cell_contents`는 `decorate`되지않은 자기자신이므로 `decorate`되지 않은 `is_power_of_two`를 `undecorate`를 통해 받을 수 있다.<br>
따라서 `undecorate(is_power_of_two)(32)`는 `decorate`되지않은 `is_power_of_two`에 숫자 32를 넘겨서 호출하는것과 동일하므로,<br>
`out_of_period`문자열이 아닌 `True`를 출력하게 된다.

### nonlocal 선언

아래의 `make_averager`는 잘못 정의되었다. 왜 그럴까?

In [51]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [52]:
avg = make_averager()
print avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`count`가 primitive type의 변수일 때 `count += 1`이 사실상 `count = count + 1`과 동일하기 때문에 문제가 발생한다. <br>
컴파일러가 `averager()`함수 내부의 `count`를 지역변수로 간주하기 때문이다. `total`도 마찬가지다. <br>
즉, `count`와 `total`은 자유변수가 아니므로 클로저에 저장되지 않는다.

이전 예제의 `series`경우에는 변수에 새로운 값을 정의하지 않고 `append()`, `len()`과 같이 메소드를 호출했기 때문에 문제가 되지 않았다. <br>
리스트가 가변형이라는 사실을 이용했을 뿐이다.

이 문제를 해결하기 위해 파이썬3은 `nonlocal`선언을 지원한다. <br>
`nonlocal`로 변수를 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유변수임을 나타낸다.
> 파이썬2는 `nonlocal`을 지원하지 않는 대신 다른 방법을 지원한다. https://www.python.org/dev/peps/pep-3104/ <br>
> 자유변수로 의도했던 변수를 가변 객체로 정의해서, 변경하고자 하는 값을 속성으로 갖게 함.

### 데코레이터 구현하기

아래 예제는 데코레이트된 함수를 호출할 때마다 현재시간, 전달된 파라미터, 반환값을 출력한다.

In [71]:
def clock(func):
    def clocked(*args):
        '''this is clocked'''
        result = func(*args)  # func은 자유변수
        arg_str = ', '.join(repr(arg) for arg in args)
        print '[%0.8fs] %s(%s) -> %r' % (time.time(), func.__name__, arg_str, result)
        return result
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    '''this is factorial'''
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print '*****', 'calling snooze(.123)'
    snooze(.123)
    print '*****', 'calling factorial(10)'
    factorial(10)

***** calling snooze(.123)
[1551005298.99530411s] snooze(0.123) -> None
***** calling factorial(10)
[1551005298.99597597s] factorial(1) -> 1
[1551005298.99613595s] factorial(2) -> 2
[1551005298.99628806s] factorial(3) -> 6
[1551005298.99642897s] factorial(4) -> 24
[1551005298.99736595s] factorial(5) -> 120
[1551005298.99793291s] factorial(6) -> 720
[1551005298.99808097s] factorial(7) -> 5040
[1551005298.99823093s] factorial(8) -> 40320
[1551005298.99837303s] factorial(9) -> 362880
[1551005298.99863100s] factorial(10) -> 3628800


위의 예제에서 `snooze`와 `factorial`은 `clock`함수에 의해 decorate되었으므로 `clocked`과 동일한 환경을 갖게 된다.<br>
이는 두 함수의 `__name__`과 `__doc__`속성도 `clocked`함수의 것으로 대체시킨다.

In [59]:
print factorial.__name__
print factorial.__doc__
print clock(factorial).__doc__

clocked
this is clocked
this is clocked


데코레이터를 위와 같이 구현하면 `decorate`당하는 함수의 `__name__`, `__doc__` 과 같은 속성을 덮어쓸 뿐만 아니라 키워드 인자를 받을 수도 없게 된다.<br>
따라서 `functools`의 `wraps`데코레이터를 통해 자유변수로 들어오는 `func`의 속성들을 클로저 함수에 복사할 수 있다.

In [72]:
import functools
import time

def clock2(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        '''this is wrapper'''
        result = func(*args, **kwargs)  # func은 자유변수
        arg_str = ', '.join(repr(arg) for arg in args)
        print '[%0.8fs] %s(%s) -> %r' % (time.time(), func.__name__, arg_str, result)
        return result
    return wrapper

@clock2
def factorial2(n):
    '''this is factorial2'''
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print '*****', 'calling factorial(10)'
    factorial2(n=10)
    
    print factorial2.__name__
    print clock2(factorial2).__doc__

***** calling factorial(10)
[1551005322.98668599s] factorial(1) -> 1
[1551005322.98673201s] factorial(2) -> 2
[1551005322.98676395s] factorial(3) -> 6
[1551005322.98680496s] factorial(4) -> 24
[1551005322.98684311s] factorial(5) -> 120
[1551005322.98687696s] factorial(6) -> 720
[1551005322.98690701s] factorial(7) -> 5040
[1551005322.98693895s] factorial(8) -> 40320
[1551005322.98697090s] factorial(9) -> 362880
[1551005322.98718810s] factorial2() -> 3628800
factorial2
this is factorial2


### 파라미터를 받는 데코레이터

일반적으로 데코레이터는 하나의 함수를 파라미터로 받지만 함수가 아닌 다른 값들을 파라미터로 받게 할 수도 있다.

In [79]:
import functools
import time

def clock_with_param(t=0):
    def decorate(func):
        @functools.wraps(func)
        def clocked3(*args, **kwargs):
            result = func(*args, **kwargs)
            arg_str = ', '.join(repr(arg) for arg in args)
            kwarg_str = ', '.join(repr(arg) for arg in kwargs)
            print '[%0.8fs] %s(%s, %s) -> %r' % (t, func.__name__, arg_str, kwarg_str, result)
            return result
        return clocked3
    return decorate

if __name__ == '__main__':
    @clock_with_param()
    def print_hi():
        print 'hi'
        
    @clock_with_param(time.time())
    def print_hi2():
        print 'hi2'
        
    print_hi()
    print_hi2()

hi
[0.00000000s] print_hi(, ) -> None
hi2
[1551005842.41019893s] print_hi2(, ) -> None


`clock_with_param()`, `clock_with_param(time.time())`을 호출하여 리턴되는 `decorate`함수에 의해 `print_hi`, `print_hi2`가 `decorate`된다. <br>
(이렇게 되면 사실 `clock_with_param`은 데코레이터가 아니라 데코레이터를 리턴하는 팩토리 함수에 가깝다.)